# feature_generation

In [1]:
import pandas as pd
from datetime import datetime

from sklearn.preprocessing import LabelEncoder
import category_encoders as ce

In [2]:
import nbimporter
from pre_processing import load_appended_dataset, separate_train_test, load_datasets

Importing Jupyter notebook from pre_processing.ipynb


### Funciones auxiliares

In [3]:
def corr(df):
    return df.corr()['precio'].sort_values(ascending=False)[1:]

In [4]:
def aniomes(anio, mes):
    if len(str(mes)) == 1:
        return int(str(anio)+'0'+str(mes))
    return int(str(anio)+str(mes))

In [5]:
def contiene_indicadores(descripcion, indicadores):
    for indicador in indicadores:
        if indicador in str(descripcion).lower():
            return 1
    return 0

In [6]:
def binning_habitaciones(x):
    if (x==-1):
        return 'unknown'
    elif x==1:
        return 'monoambiente'
    elif (x==2):
        return 'dos ambientes'
    elif (x==3):
        return 'tres ambientes'
    elif (x==4):
        return 'cuatro ambientes'
    elif (x==5):
        return 'cinco ambientes'
    else:
        return 'mas de 5 ambientes'

In [7]:
def binning_antiguedad(x):
    if (x==-1):
        return 'unknown'
    elif x==0:
        return 'new'
    elif (x<=5):
        return '0 a 5'
    elif (x<=10):
        return '5 a 10'
    elif (x<=15):
        return '10 a 15'
    elif (x<=20):
        return '15 a 20'
    elif (x<=30):
        return '20 a 30'
    elif (x<=50):
        return '30 a 50'
    else:
        return '+50'
    

In [8]:
filas_train = 240000
filas_test = 60000

# Desarollo

In [9]:
df = load_appended_dataset()

In [10]:
display(df.head(1))
print(df.shape)

,titulo,descripcion,direccion,tipodepropiedad,ciudad,provincia,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio
id,,,,,,,,,,,,,,,,,,,,
254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Avenida Division del Norte 2005,Apartamento,Benito Juárez,Distrito Federal,NaN,2.0,1.0,2.0,80.0,80.0,23533.0,2015-08-23,0,0,0,0,0,2273000.0


(300000, 20)


In [11]:
features = {}

### Variables categoricas (encoding)

In [14]:
var_categoricas = ['tipodepropiedad', 'ciudad', 'provincia']

In [15]:
# label encoding
encoder = LabelEncoder()
encoded = df[var_categoricas].apply(encoder.fit_transform).add_suffix("_labelencoded")
encoded.to_csv('./features/categoricas_labelencoded.csv')
features['categoricas_labelencoded'] = encoded.columns.values

In [16]:
# onehot 

# para tipodepropiedad
encoder = ce.OneHotEncoder()
encoded = encoder.fit_transform(df['tipodepropiedad']).add_suffix("_ohencoded")
encoded.to_csv('./features/tipodepropiedad_ohencoded.csv')
features['tipodepropiedad_ohencoded'] = encoded.columns.values

# para provincias
encoder = ce.OneHotEncoder()
encoded = encoder.fit_transform(df['provincia']).add_suffix("_ohencoded")
encoded.to_csv('./features/provincia_ohencoded.csv')
features['provincia_ohencoded'] = encoded.columns.values

# para ciudades
encoder = ce.OneHotEncoder()
encoded = encoder.fit_transform(df['ciudad']).add_suffix("_ohencoded")
encoded.to_csv('./features/ciudad_ohencoded.csv')
features['ciudad_ohencoded'] = encoded.columns.values


### Generacion de nuevas features

##### FECHA

In [12]:
df_fecha = df[['fecha', 'precio', 'metroscubiertos']].copy()

df_fecha['anio'] = df_fecha['fecha'].dt.year
df_fecha['mes'] = df_fecha['fecha'].dt.month
df_fecha['dia'] = df_fecha['fecha'].dt.day
df_fecha['timestamp'] = df_fecha['fecha'].apply(lambda x: datetime.timestamp(x))
df_fecha['aniomes'] = df_fecha.apply(lambda row: aniomes(row['anio'], row['mes']), axis=1)

In [18]:
# calculamos el precio promedio del metro cubierto por aniomes
precio_mtcubierto_aniomes = df_fecha.head(filas_train).groupby(['aniomes'])['metroscubiertos', 'precio'].agg(sum).apply(lambda x: x['precio']/x['metroscubiertos'], axis=1)
df_fecha['precio_promedio_metrocubierto_aniomes'] = df_fecha['aniomes'].apply(lambda x: precio_mtcubierto_aniomes[x])

In [19]:
# calculamos el precio promedio del metro cubierto por mes
precio_mtcubierto_mes = df_fecha.head(filas_train).groupby(['mes'])['metroscubiertos', 'precio'].agg(sum).apply(lambda x: x['precio']/x['metroscubiertos'], axis=1)
df_fecha['precio_promedio_metrocubierto_mes'] = df_fecha['mes'].apply(lambda x: precio_mtcubierto_mes[x])

In [20]:
df_fecha = df_fecha.drop(['fecha', 'precio', 'metroscubiertos'], axis=1)
pasar_a_enteros = ['precio_promedio_metrocubierto_mes', 'precio_promedio_metrocubierto_aniomes']
df_fecha[pasar_a_enteros] = df_fecha[pasar_a_enteros].astype('uint16')

In [21]:
df_fecha.to_csv('./features/fecha.csv')
features['fecha'] = df_fecha.columns.values

In [22]:
encoder = ce.OneHotEncoder()
encoded = encoder.fit_transform(df_fecha['aniomes'].astype('category')).add_suffix("_ohencoded")
encoded.to_csv('./features/aniomes_ohencoded.csv')

features['aniomes_ohencoded'] = encoded.columns.values

##### DESCRIPCION

In [23]:
df_descripcion = df[['descripcion']].copy()

indicadores = ["jardin", "patio", "jardín"]
df_descripcion["jardin"] = df_descripcion["descripcion"].map(lambda x: contiene_indicadores(x, indicadores))

indicadores = ["vigilancia", "seguridad"]
df_descripcion["vigilancia"] = df_descripcion["descripcion"].map(lambda x: contiene_indicadores(x, indicadores))

indicadores = ["aire acondicionado", "aires acondicionados"]
df_descripcion["aire_acondicionado"] = df_descripcion["descripcion"].map(lambda x: contiene_indicadores(x, indicadores))

indicadores = ["ventilador", "ventiladores"]
df_descripcion["ventilador"] = df_descripcion["descripcion"].map(lambda x: contiene_indicadores(x, indicadores))

indicadores = ["calefaccion", "caloventor", "estufa", "calefacción", "calentador"]
df_descripcion["calefaccion"] = df_descripcion["descripcion"].map(lambda x: contiene_indicadores(x, indicadores))

In [24]:
df_descripcion.drop('descripcion', inplace=True, axis=1)
df_descripcion.to_csv('./features/descripcion.csv')

features['descripcion'] = df_descripcion.columns.values

##### MEAN Y STD DE SUPERFICIE CUBIERTA POR CIUDAD

In [25]:
df_superficie_cubierta = df[['metroscubiertos','ciudad']].copy().reset_index()

avg_metroscubiertos = df_superficie_cubierta.groupby('ciudad').agg({"metroscubiertos":["mean","std"]}).reset_index()
avg_metroscubiertos.columns = ['ciudad','metroscubiertos_mean','metroscubiertos_std']

df_superficie_cubierta_mean_std = pd.merge(df_superficie_cubierta,avg_metroscubiertos,how='inner',on='ciudad').set_index('id')
df_superficie_cubierta_mean_std = df_superficie_cubierta_mean_std[["metroscubiertos_mean","metroscubiertos_std"]]

df_superficie_cubierta_mean_std.to_csv('./features/superficie_mean_std.csv')

##### BINNING (+OHE)

In [26]:
# binning para antiguedad
df_antiguedad = df[['antiguedad']].copy()

In [27]:
df_antiguedad['antiguedad'].fillna(-1, inplace=True)
df_antiguedad['antiguedad'] = df_antiguedad['antiguedad'].astype(int)
df_antiguedad['antiguedad_binning'] = df_antiguedad['antiguedad'].map(lambda x: binning_antiguedad(x))

In [28]:
# ohe
encoder = ce.OneHotEncoder()
encoded = encoder.fit_transform(df_antiguedad['antiguedad_binning']).add_suffix("_ohencoded")
encoded.to_csv('./features/antiguedad_binning.csv')
features['antiguedad_binning'] = encoded.columns.values

In [29]:
# binning para habitaciones
df_habitaciones = df[['habitaciones']].copy()

In [30]:
df_habitaciones['habitaciones'].fillna(-1, inplace=True)
df_habitaciones['habitaciones'] = df_habitaciones['habitaciones'].astype(int)
df_habitaciones['hab_binning'] = df_habitaciones['habitaciones'].map(lambda x: binning_habitaciones(x))

In [31]:
# ohe
encoder = ce.OneHotEncoder()
encoded = encoder.fit_transform(df_habitaciones['hab_binning']).add_suffix("_ohencoded")
encoded.to_csv('./features/habitaciones_binning.csv')
features['habitaciones_binning'] = encoded.columns.values

##### COTIZACION DEL USD

In [38]:
df_cotizacion = df_fecha[['aniomes']].copy().reset_index()

# Cargamos el dataset preprocesado en pre_processing.
usd = pd.read_csv('data/usd_mxn_featured')

In [39]:
usd = usd[['aniomes', 'usd_mxn', 'mxn_usd']]

In [40]:
df_cotizacion = pd.merge(df_cotizacion, usd, on='aniomes', how='inner').drop('aniomes', axis=1)

In [41]:
df_cotizacion.set_index('id', inplace=True)

In [42]:
df_cotizacion

,usd_mxn,mxn_usd
id,,
254099,16.55105,0.060419
18684,16.55105,0.060419
81195,16.55105,0.060419
14228,16.55105,0.060419
46540,16.55105,0.060419
...,...,...
131588,12.71043,0.078676
5686,12.71043,0.078676
234506,12.71043,0.078676


In [43]:
df_cotizacion.to_csv('./features/cotizacion.csv')
features['cotizacion'] = df_cotizacion.columns.values

# Full DF

In [47]:
features.keys()

dict_keys(['categoricas_labelencoded', 'tipodepropiedad_ohencoded', 'provincia_ohencoded', 'ciudad_ohencoded', 'fecha', 'aniomes_ohencoded', 'descripcion', 'antiguedad_binning', 'habitaciones_binning', 'cotizacion'])

In [48]:
def load_featured_appended_dataset():
    """Carga el dataset original, y le agrega todas las features obtenidas en este notebook. Devuelve
    un unico dataset, del cual las primeras 240.000 filas son de train.csv y las ultimas 60.000 de test.csv"""
    
    df = load_appended_dataset()
    # archivos (features.keys())
    files = ['categoricas_labelencoded', 'tipodepropiedad_ohencoded', 'provincia_ohencoded', 'ciudad_ohencoded',
             'fecha', 'aniomes_ohencoded', 'descripcion', 'antiguedad_binning', 'habitaciones_binning',
             'cotizacion']
    for file in files:
        ruta = "features/"+file+".csv"
        add = pd.read_csv(ruta, index_col='id')
        df = pd.merge(df,add,how='left',left_index=True, right_index=True)
        
    return df

In [49]:
def load_featured_datasets():
    """Carga el dataset original, y le agrega todas las features obtenidas en este notebook. Devuelve
    dos datasets: train, test."""
    
    df = load_featured_appended_dataset()
    
    return separate_train_test(df)

# Resultados

In [31]:
features
# hardcodear diccionario para devolverlo

{'categoricas_labelencoded': array(['tipodepropiedad_labelencoded', 'ciudad_labelencoded',
        'provincia_labelencoded'], dtype=object),
 'tipodepropiedad_ohencoded': array(['tipodepropiedad_1_ohencoded', 'tipodepropiedad_2_ohencoded',
        'tipodepropiedad_3_ohencoded', 'tipodepropiedad_4_ohencoded',
        'tipodepropiedad_5_ohencoded', 'tipodepropiedad_6_ohencoded',
        'tipodepropiedad_7_ohencoded', 'tipodepropiedad_8_ohencoded',
        'tipodepropiedad_9_ohencoded', 'tipodepropiedad_10_ohencoded'],
       dtype=object),
 'provincia_ohencoded': array(['provincia_1_ohencoded', 'provincia_2_ohencoded',
        'provincia_3_ohencoded', 'provincia_4_ohencoded',
        'provincia_5_ohencoded', 'provincia_6_ohencoded',
        'provincia_7_ohencoded', 'provincia_8_ohencoded',
        'provincia_9_ohencoded', 'provincia_10_ohencoded',
        'provincia_11_ohencoded', 'provincia_12_ohencoded',
        'provincia_13_ohencoded', 'provincia_14_ohencoded',
        'provincia_15_